# Introduction

The purpose of this project is to model COVID-19 daily reporting data from around the world into a star shema in order to actively track the course of the pandemic. The data ingestion will be controlled by Airflow with the data residing in an Amazon Redshift MPP data warehouse. The purpose of this notebook is to get familiar with the daily covid data provided by Johns Hoplins University (https://github.com/CSSEGISandData/COVID19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports).



# Exploring the Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/07-27-2020.csv')

This is data from 7-26-20

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

This is good news. Alas, JHU has already done some major data cleaning.
The field Country_Region, which is the country, doesn't appear to be null. It seems that Province States can be NULL as some countries do not have robust enough reporting to break down the data. 

Let's see if we can find an example of this. 

In [ ]:
df[df['Province_State'].isnull()].sample(5)

Interesting that France is not reporting region level data in this report. Let's dig in further

In [ ]:
df[df['Country_Region'] == 'France']

Hmm, it looks like there are numerous territories under France represented here. Not a problem!

Let's see how many countries are represented

In [ ]:
len(df['Country_Region'].unique()) #188 countries represented, what are they?

In [ ]:
for country in df['Country_Region'].unique():
    print(country)

How many states/provinces per country? Which one has the most?


In [ ]:
df.groupby('Country_Region')['Province_State'].nunique().sort_values(ascending=False).head(20)

Wow, Russia beats out the US with Japan a close 3rd. Now that we have a good sense of the countries/regions represented...let's explore a little the case # data.

In [ ]:
df.groupby('Country_Region').sum()['Confirmed'].sort_values(ascending=False).head(10) 

In [ ]:
df.groupby('Country_Region').sum()['Deaths'].sort_values(ascending=False).head(10) 

In [ ]:
df.groupby('Country_Region').sum()['Recovered'].sort_values(ascending=False).head(10) 

In [ ]:
df.groupby('Country_Region').mean()['Incidence_Rate'].sort_values(ascending=False).head(10) 

In [ ]:
df.groupby('Country_Region').mean()['Case-Fatality_Ratio'].sort_values(ascending=False).head(10) 

I think we have a good sense of the World Data, lets dig into how we might want to model this.

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

As far as modeling, I think there are going to be 5 main tables in our Star Schema.

dim_city
    -based on the Admin2 field, many NULLS as not every Province/Country has a corresponding "city"
 
dim_state
    - based on the Province_State field

dim_country
    - based on the Country_Region field 

dim_coordinates
    - based on the lat/long coordinates of each row
    - use the combined key to match coordinates

    
fact_covid 
    - will contain measures for confirmed, active, recovered, incidence_rate, case_fatality_ratio
    - Will use the timestamp column last_updated and output the date as YYYY-MM-DD
    
All other fields will be discarded from the raw data as they are not necessary to conduct our Analysis.

# Data Transformation Discovery Work

For the next section of the notebook, we will load a few days worth of data into a sqlite db in order to practice writing the necessary SQL for our transformations. The data flow will include a staging_raw table which is going to be an exact dump of data from the JHU github repo. So, this sqlite db will serve as a good practive environment for writing the necessary SQL statements in order to transform raw data into our star schema mentioned above. 

In [ ]:
gh_link = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'

In [ ]:
dates = ['07-24-2020','07-25-2020','07-26-2020']
df_list = []
for day in dates:
    df_list.append(pd.read_csv(gh_link.format(day)))
    
df = pd.concat(df_list)
    

In [ ]:
df.shape

In [ ]:
from sqlalchemy import create_engine


In [ ]:
engine = create_engine('sqlite://', echo=False)


In [ ]:
df.to_sql('staging_raw_covid', con=engine)

In [ ]:
pd.read_sql_query('SELECT * FROM staging_raw_covid LIMIT 5', con=engine) # ok, that worked, let's get to modeling

### Dim Country

In [ ]:
sql = '''

SELECT
    DISTINCT Country_Region AS country
FROM staging_raw_covid

'''



In [ ]:
pd.read_sql_query(sql, con=engine) #great

In [ ]:
df = pd.read_sql_query(sql, con=engine)
df.to_sql('dim_country', con=engine, index_label='country_id', if_exists='replace')

In [ ]:
pd.read_sql_query('SELECT * FROM dim_country', con=engine)

### Dim State

In [ ]:
sql = '''

SELECT
    DISTINCT Province_State AS state
FROM staging_raw_covid 

'''

In [ ]:
pd.read_sql_query(sql, con=engine) #great

In [ ]:
df = pd.read_sql_query(sql, con=engine)
df.to_sql('dim_state', con=engine,index_label='state_id', if_exists='replace')

In [ ]:
pd.read_sql_query('SELECT * FROM dim_state', con=engine)

## Dim City

In [ ]:
sql = '''

SELECT 
    DISTINCT Admin2 AS city
FROM staging_raw_covid

'''


In [ ]:
pd.read_sql_query(sql, con=engine) #great

In [ ]:
df = pd.read_sql_query(sql, con=engine)
df.to_sql('dim_city', con=engine, index_label='city_id', if_exists='replace')

In [ ]:
pd.read_sql_query('SELECT * FROM dim_city', con=engine)

## Dim Coordinates

In [ ]:
sql = '''

SELECT
   DISTINCT Combined_Key AS location,
     Lat AS latitude,
     Long_ AS longitude
FROM staging_raw_covid

'''

In [ ]:
pd.read_sql_query(sql, con=engine) #great

In [ ]:
df = pd.read_sql_query(sql, con=engine)
df.to_sql('dim_coordinates', con=engine, if_exists='replace', index=True, index_label = 'coordinate_id')

In [ ]:
pd.read_sql_query('SELECT * FROM dim_coordinates', con=engine)

## Fact Covid

In [ ]:
df = pd.read_sql_query('SELECT * FROM staging_raw_covid', con=engine)

In [ ]:
df.columns

In [ ]:
pd.to_datetime(df['Last_Update'])

In [ ]:
df.head()

In [ ]:
sql = '''

SELECT
    DATE(s.Last_Update) AS date,
    ci.city_id,
    st.state_id,
    c.country_id,
    coor.coordinate_id,
    s.Confirmed,
    s.Deaths,
    s.Recovered,
    s.Active,
    s.Incidence_Rate
FROM staging_raw_covid s
LEFT JOIN dim_country c ON c.country=s.Country_Region
LEFT JOIN dim_state st ON st.state=s.Province_State
LEFT JOIN dim_city ci ON ci.city=s.Admin2
LEFT JOIN dim_coordinates coor ON coor.location=s.Combined_key

'''

In [ ]:
pd.read_sql_query(sql, con=engine) #great

We now have a good idea of how to model our data. The actual SQL used to create the star schema may be a little different as we had to imporovise in our sqlite/pandas environment, however the basic ideas will remain the same!!